In [1]:
import networkx as nx
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import re
import datetime
import time

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
init_notebook_mode(connected=True)

import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch

import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline

import progressbar

import seaborn as sns

predtict graph theory to max profit
  by predit
  
  
  

In [2]:
######## Parameters #########

time_zone_interval = 60*8

######## Parameters #########



######## Function Start ########
def time_rounding(x, base=5):
    return int(base * round(float(x)/base))


def dropoff_prob_matrix(arr):
    '''compute the prbability of dropoff
    given the picuk zones
    return a matrix with row:pickup
    col: probility of dropoff'''
    total_pickups = arr.sum(axis=1)
    dim = arr.shape
    result_arr = np.zeros(dim)
    for i in range(dim[0]):
        for j in range(dim[1]):
            if total_pickups[i] == 0:
                continue
            elif arr[i,j] == 0:
                continue
            else:
                result_arr[i,j] = arr[i,j]/total_pickups[i]
    return result_arr



def Greedy_Algorithm(start_zone):
    '''Greedy algorithm search the at each states
    the minimized moving & wait time zone.
    return a path, total_trip_time & total_waste'''
    
      
    ## loading data
    matrix_PU_time= pd.read_csv('./Matrix/matrix_PU_time.csv',index_col=0)
    matrix_PU_time.columns =matrix_PU_time.columns.astype(float)

    matrix_PU_time.fillna(value=8*60, inplace=True)
    matrix_PU_wait_time = matrix_PU_time.set_index(matrix_PU_time.index.astype(int)).sort_index()
    
    matrix_zone_distance = pd.read_csv('./Matrix/full_zone_distance_matrix.csv',index_col=0)
    matrix_zone_distance.columns =matrix_zone_distance.columns.astype(int)
    matrix_zone_distance.index = matrix_zone_distance.index + 1
    temp_index = matrix_zone_distance.index
    matrix_zone_distance = matrix_zone_distance.T
    matrix_zone_distance.index = temp_index
    ind_dis = set(matrix_zone_distance.iloc[146].index)
    ind_wait_time = set(matrix_PU_wait_time[0].index)
    redundant_zones = list(ind_dis.difference(ind_wait_time))
    matrix_zone_distance = matrix_zone_distance.drop(redundant_zones, axis=1).drop(list(map(int, redundant_zones)))
    matrix_zone_distance[matrix_zone_distance.isna().any(axis=1)]
    
    matrix_prob_PU_DO_t0 = pd.read_csv('./Matrix/matrix_prob_PU_DO_t0.csv', index_col=0)
    matrix_prob_PU_DO_t0.index = matrix_prob_PU_DO_t0.index.astype(int)
    matrix_prob_PU_DO_t0.columns = matrix_prob_PU_DO_t0.columns.astype(int)
    top_dropoff_zone_t0 = pd.DataFrame(matrix_prob_PU_DO_t0.idxmax(axis=1))
    
    
    top_dropoff_zone_t1 = pd.DataFrame(matrix_prob_PU_DO_t1.idxmax(axis=1))
    
    result_path = list() # path with zone as node
    total_trip_time = 0 # total time that are making $
    total_waste_time = 0 # total time waste in move & wait
    total_time = 24*60 # time of a day
    temp_zone = start_zone # set current zone to starting zone
    
    while total_time >= 0:
        result_path.append(temp_zone)
        # compute highest pro drop off zone
        dropoff_zone = top_dropoff_zone_t1.iloc[temp_zone][0]
        result_path.append(dropoff_zone)
        
        # compute trip_time between pickup and dropoff zone
        dropoff_trip_time =  matrix_zone_distance.iloc[temp_zone,dropoff_zone]
        total_trip_time = total_trip_time + dropoff_trip_time # add trip time to total
        total_time = total_time - dropoff_trip_time # subtract the trip time from the day
        
        # compute moving_time to other zones and the wait time in other zones
        trip_wait_time_table = pd.DataFrame(np.array(matrix_zone_distance.iloc[260])
                                            + np.array(matrix_PU_wait_time[0])).sort_values(by=0)

        # looking for the min 
        local_min_cost_zone = trip_wait_time_table.idxmin()[0]
        local_min_cost_time = trip_wait_time_table.min()[0]
        
        # update current(temp) zone, total_waste time on moving and waiting, and day
        temp_zone = local_min_cost_zone
        total_waste_time = total_waste_time + local_min_cost_time
        total_time = total_time - local_min_cost_time
        
    return result_path, total_trip_time, total_waste_time




def Randomized_Greedy_Algorithm(start_zone, randomness=3):

    '''Randomized Greedy algorithm search the at each states
    the minimized moving & wait time zone. with some randomness
    return a path, total_trip_time & total_waste
    
    Args:
        start_zone(int): a int represent TLC zones as
        a starting zone of the alg
        
        randomness(float): between 0 and 1 to given some 
        randomness like entropy function, but with 1 the most
        random and 0 == Greedy Alg
        
    Retrun:
        result_list(list): path of the taxi in a day
        
        total_trip_time(float): total time that taxi is making 
            money i.e. serving customers
        
        total_waste_time(float): time moving and waiting

        '''
    
    
    ## loading data
    matrix_PU_time= pd.read_csv('./Matrix/matrix_PU_time.csv',index_col=0)
    matrix_PU_time.columns =matrix_PU_time.columns.astype(float)

    matrix_PU_time.fillna(value=8*60, inplace=True)
    matrix_PU_wait_time = matrix_PU_time.set_index(matrix_PU_time.index.astype(int)).sort_index()
    
    matrix_zone_distance = pd.read_csv('./Matrix/full_zone_distance_matrix.csv',index_col=0)
    matrix_zone_distance.columns =matrix_zone_distance.columns.astype(int)
    matrix_zone_distance.index = matrix_zone_distance.index + 1
    temp_index = matrix_zone_distance.index
    matrix_zone_distance = matrix_zone_distance.T
    matrix_zone_distance.index = temp_index
    ind_dis = set(matrix_zone_distance.iloc[146].index)
    ind_wait_time = set(matrix_PU_wait_time[0].index)
    redundant_zones = list(ind_dis.difference(ind_wait_time))
    matrix_zone_distance = matrix_zone_distance.drop(redundant_zones, axis=1).drop(list(map(int, redundant_zones)))
    matrix_zone_distance[matrix_zone_distance.isna().any(axis=1)]
    
    matrix_prob_PU_DO_t0 = pd.read_csv('./Matrix/matrix_prob_PU_DO_t0.csv', index_col=0)
    matrix_prob_PU_DO_t0.index = matrix_prob_PU_DO_t0.index.astype(int)
    matrix_prob_PU_DO_t0.columns = matrix_prob_PU_DO_t0.columns.astype(int)
    
    top_dropoff_zone_t1 = pd.DataFrame(matrix_prob_PU_DO_t1.idxmax(axis=1))
    
    
    random_number = int(-0.2/(.000000000000001-(randomness+0.000001)) +0.812)
    if random_number > 20:
        random_number = 20
    elif random_number < 1:
        random_number = 1
    result_path = list() # path with zone as node
    total_trip_time = 0 # total time that are making $
    total_waste_time = 0 # total time waste in move & wait
    total_time = 24*60 # time of a day
    temp_zone = start_zone # set current zone to starting zone
    
    while total_time >= 0:
        result_path.append(temp_zone)
        # compute highest pro drop off zone
        dropoff_zone = top_dropoff_zone_t1.iloc[temp_zone][0]
        result_path.append(dropoff_zone)
        
        # compute trip_time between pickup and dropoff zone
        dropoff_trip_time =  matrix_zone_distance.iloc[temp_zone,dropoff_zone]
        total_trip_time = total_trip_time + dropoff_trip_time # add trip time to total
        total_time = total_time - dropoff_trip_time # subtract the trip time from the day
        
        # compute moving_time to other zones and the wait time in other zones
        trip_wait_time_table = pd.DataFrame(np.array(matrix_zone_distance.iloc[260])
                                            + np.array(matrix_PU_wait_time[0])).sort_values(by=0)

        # looking for the min 
        trip_wait_time_table = trip_wait_time_table[:20].sample(random_number)
        local_min_cost_zone = trip_wait_time_table.idxmin()[0]
        local_min_cost_time = trip_wait_time_table.min()[0]
        
        # update current(temp) zone, total_waste time on moving and waiting, and day
        temp_zone = local_min_cost_zone
        total_waste_time = total_waste_time + local_min_cost_time
        total_time = total_time - local_min_cost_time
        
    return result_path, total_trip_time, total_waste_time



class Exponential:

    def __init__(self, rate):
        self.rate = rate

    def prob_less_than_or_equal(self, t):
        rate = self.rate * t
        return 1 - np.exp(-rate)

    def prob_greater_than(self, t):
        return 1 - self.prob_less_than_or_equal(t)

    def prob_between(self, t1, t2):
        p1 = self.prob_less_than_or_equal(t1)
        p2 = self.prob_less_than_or_equal(t2)

        return p2 - p1


expo = Exponential(2)
print(expo.prob_less_than_or_equal(0.25))
print(expo.prob_between(0.25, 0.5))


######## Function End ########

0.3934693402873666
0.2386512185411911


In [3]:
%%time
yellow = pd.read_csv('./Data/yellow.csv')
yellow.head()

<string>:2: DtypeWarning:

Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.



CPU times: user 20.7 s, sys: 2.36 s, total: 23.1 s
Wall time: 23.1 s


In [4]:
# create timezone with interval 15min
yellow['Time_Diff'] = pd.to_datetime(yellow['tpep_dropoff_datetime']) - pd.to_datetime(yellow['tpep_pickup_datetime'])
yellow_new = yellow.iloc[:7781158,]
yellow_new.shape
yellow_new['Time_Diff'].mean()
yellow_new['dropoff_datetime'] = yellow_new['tpep_dropoff_datetime'].str[11:13].astype(float) * 60 + yellow_new['tpep_dropoff_datetime'].str[14:16].astype(float) + yellow_new['tpep_dropoff_datetime'].str[17:19].astype(float) / 60
yellow_new['pickup_datetime'] = yellow_new['tpep_pickup_datetime'].str[11:13].astype(float) * 60 + yellow_new['tpep_pickup_datetime'].str[14:16].astype(float) + yellow_new['tpep_pickup_datetime'].str[17:19].astype(float) / 60
yellow_new['dropoff_timezone'] = (yellow_new['dropoff_datetime'] //time_zone_interval)
yellow_new['pickup_timezone'] = (yellow_new['pickup_datetime'] // time_zone_interval)

yellow_new['pickup_date'] = pd.to_datetime(yellow_new['tpep_pickup_datetime']).dt.date
yellow_new['dropoff_date'] = pd.to_datetime(yellow_new['tpep_dropoff_datetime']).dt.date
yellow_new = yellow_new[yellow_new['pickup_date']==yellow_new['dropoff_date']]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [5]:
yellow_new.shape

(7696006, 27)

In [6]:
loc_ids = list(range(266))
num_loc_ids = len(loc_ids)

time_zones = list(yellow_new['pickup_timezone'].unique())
time_zones = sorted(time_zones)
num_time_zones = len(time_zones)
print(num_loc_ids)
print(num_time_zones)

266
3


In [7]:
## create 2-d matrix to with each row as PULocationID and each col as the time_zone
matrix_PU_time = pd.DataFrame() # create empty dataframe

for time_zone in  time_zones:
    temp_df = (time_zone_interval / (yellow_new[yellow_new['pickup_timezone']==time_zone]
                                    .groupby(['PULocationID'])['DOLocationID']
                                    .agg(['count'])))
    temp_df.rename(index=str, columns={"count": time_zone},inplace=True)
    matrix_PU_time = matrix_PU_time.merge(temp_df,
                                                how='outer',
                                                left_index = True,
                                                right_index = True)
    
matrix_PU_time.shape

(262, 3)

In [8]:
matrix_PU_wait_time

NameError: name 'matrix_PU_wait_time' is not defined

In [ ]:
matrix_PU_time.to_csv('./Matrix/matrix_PU_time.csv')

In [ ]:
matrix_PU_time.fillna(value=time_zone_interval, inplace=True)
matrix_PU_wait_time = matrix_PU_time.set_index(matrix_PU_time.index.astype(int)).sort_index()

In [ ]:
arr_time_PU_DO = np.zeros([num_time_zones,num_loc_ids,num_loc_ids])
print(('1d: each time_zone'
      +'\n2d: each pick up zone'
      +'\n3d: each dropoff zone'))
arr_time_PU_DO.shape

In [ ]:
%%time
bar = progressbar.ProgressBar(max_value=num_time_zones-1)
for time_zone in range(num_time_zones):
    
    bar.update(time_zone)# print out the progress and running time
    
    temp_PU_DO = (yellow_new[yellow_new['pickup_timezone']==time_zone]
               .groupby(['PULocationID'])['DOLocationID']
               .value_counts()).unstack(fill_value=0)

    temp_arr = np.zeros([num_loc_ids,num_loc_ids])
    for i in range(num_loc_ids): 
        for j in range(num_loc_ids):
            try:
                temp_arr[i,j] = temp_PU_DO.loc[i,j]
            except:
                continue
    arr_time_PU_DO[time_zone] =  temp_arr

In [ ]:
arr_time_PU_DO.shape

In [ ]:
## create a probability array that given a time zone,  it have row as a pickup loc and col as prob(dropoff loc)
arr_prob_PU_DO = np.zeros(arr_time_PU_DO.shape)  # create a 3-d array of the same shape of arr_time_PU_DO
for time_zone in range(num_time_zones):
    arr_prob_PU_DO[time_zone] = dropoff_prob_matrix(arr_time_PU_DO[time_zone])

In [ ]:
## loading the distance matrix(compute from the full_yellow.csv)
M = 1000000000000 # large number
zone_dis = pd.read_csv('./Matrix/zone_distance_matrix.csv', index_col=0)

In [ ]:
matrix_zone_distance = pd.read_csv('./Matrix/full_zone_distance_matrix.csv',index_col=0)
matrix_zone_distance.index = matrix_zone_distance.index + 1
temp_index = matrix_zone_distance.index
matrix_zone_distance = matrix_zone_distance.T
matrix_zone_distance.index = temp_index
matrix_zone_distance
matrix_zone_distance.shape

In [ ]:
ind_dis = set(matrix_zone_distance.iloc[146].index)
ind_wait_time = set(matrix_PU_wait_time[0].index)
redundant_zones = list(ind_dis.difference(ind_wait_time))
redundant_zones
matrix_zone_distance = matrix_zone_distance.drop(redundant_zones, axis=1).drop(list(map(int, redundant_zones)))
matrix_zone_distance[matrix_zone_distance.isna().any(axis=1)]
matrix_zone_distance.shape

In [ ]:
matrix_zone_distance = pd.read_csv('./Matrix/full_zone_distance_matrix.csv',index_col=0)
matrix_zone_distance.index = matrix_zone_distance.index + 1
temp_index = matrix_zone_distance.index
matrix_zone_distance = matrix_zone_distance.T
matrix_zone_distance.index = temp_index
ind_dis = set(matrix_zone_distance.iloc[146].index)
ind_wait_time = set(matrix_PU_wait_time[0].index)
redundant_zones = list(ind_dis.difference(ind_wait_time))
matrix_zone_distance = matrix_zone_distance.drop(redundant_zones, axis=1).drop(list(map(int, redundant_zones)))
matrix_zone_distance[matrix_zone_distance.isna().any(axis=1)]


In [ ]:
matrix_zone_distance
matrix_PU_wait_time
arr_prob_PU_DO.shape

In [ ]:
matrix_prob_PU_DO_t0 = pd.DataFrame(arr_prob_PU_DO[0])
matrix_prob_PU_DO_t1 = pd.DataFrame(arr_prob_PU_DO[1])
matrix_prob_PU_DO_t2 = pd.DataFrame(arr_prob_PU_DO[2])

In [ ]:
matrix_prob_PU_DO_t0 = pd.read_csv('./Matrix/matrix_prob_PU_DO_t0.csv')

In [ ]:
matrix_prob_PU_DO_t0.iloc[104]

In [ ]:
matrix_zone_distance.iloc[104,0]

In [ ]:

pd.DataFrame(np.array(matrix_zone_distance.iloc[260]) + np.array(matrix_PU_wait_time[0])).sort_values(by=0).min()


In [ ]:
top_dropoff_zone_t1 = pd.DataFrame(matrix_prob_PU_DO_t1.idxmax(axis=1))
top_dropoff_zone_t1

In [ ]:
top_dropoff_zone_t1 = pd.DataFrame(matrix_prob_PU_DO_t1.idxmax(axis=1))
top_dropoff_zone_t1.iloc[98][0]

In [ ]:
path = Randomized_Greedy_Algorithm(66, 1)[0]

In [ ]:
from taxi_graph import graph_path
graph_path(path)

In [ ]:
matrix_rounded_zone_distance = pd.DataFrame()
for col in matrix_zone_distance.columns:
    matrix_rounded_zone_distance[col] = matrix_zone_distance[col].apply(time_rounding)
matrix_rounded_zone_distance = matrix_rounded_zone_distance.replace(0,5)

In [ ]:
matrix_rounded_zone_distance

In [ ]:
matrix_nodes

In [ ]:
G_round = nx.DiGraph()
matrix_nodes = pd.DataFrame()
for col in [1,2,3,4]:
        matrix_nodes[col] = [0 for i in range(5,12*5,5)]
matrix_nodes.index = list(range(5,12*5,5))

for col in [1,2,3,4]:
    for row in range(5,12*5,5):
        matrix_nodes[col].loc[row] = str(col)+','+str(row)
        G_round.add_node((col,row),pos=(col,-row))


In [ ]:
matrix_nodes

In [ ]:
matrix_zone_distance.loc[1]

In [ ]:
G_round.nodes()
G_round.add_edge((1,5), (1,10))
G_round.edges()

In [ ]:
matrix_roun

In [ ]:
count_row = 0
count = 0
for row in matrix_nodes.index:
    count_row += 1
    count_col = 0
    for col in matrix_nodes.columns:
        count_col += 1
        if matrix_round_zone_distance[row] 
        G_round.add_edge(col)


In [ ]:
plt.figure(1,figsize=(22,33))

pos = nx.get_node_attributes(G_round, 'pos')
# pos = nx.spring_layout(G_round)
nx.draw_networkx_nodes(G_round,pos,node_size=10,font_size=7,width=8,alpha=1.0)
nx.draw_networkx_edges(G_round,pos,font_size=7,edgelist=G_round.edges(),
                       width=2,alpha=0.5,edge_color='b',edge_size=2)


nx.draw_networkx_labels(G_round, pos)
# nx.draw(G_round, pos, with_labels=True, node_size=0)
plt.axis('on')
plt.show()

In [ ]:
## zone to zone moving distance in time mins
matrix_zone_distance.iloc[217,256]

In [ ]:
matrix_prob_PU_DO_t0[].idxmax()

In [ ]:
matrix_PU_wait_time[0][18]

In [ ]:
current_zone = 147
pd.DataFrame(np.array(matrix_zone_distance.iloc[current_zone])
                                            + np.array(matrix_PU_wait_time[0])).sort_values(by=0)
